In [ ]:
import pandas as pd
import processor as pr
import loader as load
ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)


In [ ]:

# print(stad_ge_phylum_stage["stage"].values)
stage_dictionary = {
    "I":0,
    "IA":0,
    "IB":0,
    "II":1,
    "IIA":1,
    "IIB":1,
    "III":2,
    "IIIA":2,
    "IIIB":2,
    "IIIC":2,
    "IV":3,
}

def convertStage(stage):
    st = str(stage)[6:]
    return stage_dictionary[st]

stad_ge_phylum_stage = ge_phylum_stage[ge_phylum_stage["project"]=="STAD"]
stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)   
print(stad_ge_phylum_stage["stage"].value_counts())
stad_ge_phylum_stage = stad_ge_phylum_stage.dropna(subset=["stage"])
# load.attachTumorStatus(stad_ge_phylum_stage)
x, y = pr.splitData(stad_ge_phylum_stage, "stage", project="STAD")

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial').fit(x, y)
print(model.predict(x))
print(y.values)

In [ ]:
import predictor as pred

tcma_genus = load.loadTCMA("genus")
tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus")
# aak_ge = load.loadGEWithClinical()
aak_ge = None
data = [tcma_genus, aak_ge, tcma_genus_aak_ge]
files = ["tcma_gen", "aak_ge", "tcma_gen_aak_ge"]

In [ ]:
for i, d in enumerate(data[:1]):
    load.attachTumorStatus(d)
    final_report = ""
    for c in ["COAD", "ESCA", "HNSC", "READ", "STAD"]:   
        if i == 2 and  c == "READ":
            continue
        x, y = pr.splitData(d, target="tumor", project=c)
        cur_report = pred.runCrossValidation(x, y)
        final_report += f"Summary data for: {c}\n"
        description = ",".join([k + ": " + str(cur_report[k]) for k in cur_report])
        final_report += description + "\n"
        load.saveDescriptor(final_report, fr'Data\Descriptor\Prediction\{files[i]}_tmr_pred.txt')
# print("Final final_report", final_report)
# Warnings because everything is being predicted as the same
